# Notes for current output

_(Notes about currently visible output in the notebook.)_

The reduction method used was for all conductances initialized to their area-weighted average values, and scaled using the surface area ratio. No other fitting steps were done (not even matching input impedance for each subtree). The dictionary with reduction parameters was as follows:

```python
{
    'Z_freq' :              25.,
    'Z_init_func' :         reduction.init_cell_steadystate,
    'Z_linearize_gating' :  False,
    'f_lambda':             100.0,
    'syn_scale_method' :    'Ai_syn_to_soma',
    'syn_position_method':  'root_distance_micron',
    'gbar_init_method':     'area_weighted_average',
    'gbar_scale_method':    'surface_area_ratio',
    'passive_scale_method': 'surface_area_ratio',
    'split_criterion':      'micron_distance',
    'split_dX':             50.0,
}
```

# Reproducibility info

Information required to reproduce results of this notebook

In [ ]:
# Enable interactive plots with backend 'notebook'
%matplotlib notebook

# print code version (hash of checked out version)
!git log -1

# print date and time of script execution
import datetime
print("\nNotebook executed at at {} in following directory:".format(datetime.datetime.now()))
%cd ..

# Original STN model

In [ ]:
from bgcellmodels.models.STN.GilliesWillshaw import gillies_model
from bgcellmodels.common import treeutils, electrotonic
from neuron import h
import matplotlib.pyplot as plt

# instantiate STN cell model
soma, dends, stims = gillies_model.stn_cell_gillies()

# get terminal segments
leaves = treeutils.leaf_sections()
root = treeutils.root_section(soma)

## Do Measurements

In [ ]:
# get distance functions
distance_funcs = {
    'Ztransfer': electrotonic.measure_transfer_impedance,
    'Ai': electrotonic.measure_current_transfer,
    'Av': electrotonic.measure_voltage_transfer,
    'Zin': electrotonic.measure_input_impedance,
}

measures_ylims = {
    'Ztransfer': (40, 70),
    'Ai': (0.5, 1.05),
    'Av': (0, 1.1),
    'Zin': (0, 1000.0),
}

In [ ]:
# Do measurement
leaf_distance_measures = electrotonic.measure_along_paths(root, leaves, distance_funcs, freq=25.0)

## Plot cell topology

In [ ]:
import bgcellmodels.morphology.morph_gui as morphology
from mpl_toolkits.mplot3d import Axes3D

plt.figure()
ax = plt.subplot(111, projection='3d')
ax.view_init(90,90) # set azimut and elevation
ax.set_zlim((-1,1)) # prevent rounding errors
ax.set_axis_off() # hide x,y,z axes and grid

# Plot shape of our cell
h.define_shape(sec=soma)
morphology.shapeplot(h, ax)

# Now mark location of leaves
marker_colors = 'rgby'
for i, leaf_sec in enumerate(leaves[:2]):
    morphology.mark_locations(h, leaf_sec, 1.0, 
                            label=leaf_sec.name(),
                            markspec='o'+marker_colors[i])

plt.legend()
plt.show()

## Plot Measurements

In [ ]:
plotted_measures = distance_funcs.keys()
fig, axes = plt.subplots(len(plotted_measures))

for i, measure in enumerate(plotted_measures):
    ax = axes[i]
    
    # Plot the same measure for each leaf in the same axis
    for j, leaf in enumerate(leaves[:2]):
        
        ax.plot(
            leaf_distance_measures[j]['pathlen_micron'],
            leaf_distance_measures[j][measure],
            label=leaf.name())
        
        ax.set_ylim(measures_ylims[measure])
        ax.legend() # show line labels
        ax.grid(True)
        ax.set_ylabel(measure)
    

# Reduced STN model

Build the Gillies & Willshaw STN model

In [ ]:
# NEW REDUCTION CODE
from bgcellmodels.models.STN.GilliesWillshaw.reduced import cersei_reduce
cersei_reduce.fold_bush(export_locals=False)

# OLD REDUCTION CODE
# import reducemodel.reduce_cell as legacy_reduce
# legacy_reduce.fold_gillies_marasco(export_locals=False)

# get terminal segments
soma = h.SThcell[0].soma
leaves = treeutils.leaf_sections(soma)
root = treeutils.root_section(soma)

## Plot topology

In [ ]:
# import bgcellmodels.morphology.morph_gui as morphology
# from mpl_toolkits.mplot3d import Axes3D

plt.figure()
ax = plt.subplot(111, projection='3d')
ax.view_init(90,90) # set azimut and elevation
ax.set_zlim((-1,1)) # prevent rounding errors
ax.set_axis_off() # hide x,y,z axes and grid

# Plot shape of our cell
h.define_shape(sec=soma)
morphology.shapeplot(h, ax)

# Now mark location of leaves
marker_colors = 'rgby'
for i, leaf_sec in enumerate(leaves[:2]):
    morphology.mark_locations(h, leaf_sec, 1.0, 
                            label=leaf_sec.name(),
                            markspec='o'+marker_colors[i])

plt.legend()
plt.show()

## Plot measurements

In [ ]:
# Do measurement
leaf_distance_measures_old = leaf_distance_measures
leaf_distance_measures_new = electrotonic.measure_along_paths(root, leaves, distance_funcs, freq=25.0)

In [ ]:
plotted_measures = distance_funcs.keys()
fig, axes = plt.subplots(len(plotted_measures))

for i, measure in enumerate(plotted_measures):
    ax = axes[i]
    
    # Plot the same measure for each leaf in the same axis
    for j, leaf in enumerate(leaves[:2]):
        
        ax.plot(
            leaf_distance_measures_new[j]['pathlen_micron'],
            leaf_distance_measures_new[j][measure],
            label=leaf.name())
        
        ax.plot(
            leaf_distance_measures_old[j]['pathlen_micron'],
            leaf_distance_measures_old[j][measure],
            'r--', linewidth=.5)
        
        ax.set_ylim(measures_ylims[measure])
        ax.legend() # show line labels
        ax.grid(True)
        ax.set_ylabel(measure)

## ARTICLE figure

In [ ]:
def get_style_colors():
    """
    Get colors of the currently active style.
    """
    return plt.rcParams['axes.prop_cycle'].by_key()['color']

plotutils.get_style_colors = get_style_colors

In [ ]:
from bgcellmodels.common import plotutils

plotted_measures = ['Zin', 'Ztransfer']
measures_ylims['Ztransfer'] = (40, 70)
measures_ylims['Zin'] = (0, 1300)

fig, axes = plt.subplots(len(plotted_measures), figsize=(8.0, 8.0))

for i, measure in enumerate(plotted_measures):
    ax = axes[i]
    
    # Plot the same measure for each leaf in the same axis
    for j, leaf in enumerate(leaves[:2]):
        
        ax.plot(
            leaf_distance_measures_old[j]['pathlen_micron'],
            leaf_distance_measures_old[j][measure],
            '--', marker='.', linewidth=1.0, color=plotutils.get_style_colors()[0],
            label='full' if j==0 else None)
        
        ax.plot(
            leaf_distance_measures_new[j]['pathlen_micron'],
            leaf_distance_measures_new[j][measure],
            '--', marker='.', linewidth=1.0, color=plotutils.get_style_colors()[1],
            label='reduced' if j==0 else None)
        
        ax.set_ylim(measures_ylims[measure])
        ax.legend() # show line labels
        ax.grid(True)
        ax.set_ylabel('{} (M$ \Omega $)'.format(measure))
        ax.set_xlabel('soma distance ($\mu m$)')

plotutils.save_figure(
    'stn-full-vs-red_Zin-Ztr', fig=fig,
    dir='/home/luye/cloudstore_m/notes/publications_lkmn/ARTICLE_reduction/figures')